In [ ]:
!pip install onnxmltools==1.7.0 onnxruntime==1.4.0
!pip install tensorflow==1.14.0

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from PIL import Image
import os

os.environ['TF_KERAS'] = '1'
import onnxmltools
from skl2onnx.common.data_types import FloatTensorType
import onnx
import onnxruntime
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
#1yZS6I7-j19HxVXuwD-2cOLv-SxDoGFsk
#11VhID0bod6neOfsUBkaYvkiEMVyr9Wjx
!gdown --id 1fgyvtdg6ShgXBvTZWQEBJGjKTjRATbSj
!gdown --id 11VhID0bod6neOfsUBkaYvkiEMVyr9Wjx
!gdown --id 1yZS6I7-j19HxVXuwD-2cOLv-SxDoGFsk

Downloading...
From: https://drive.google.com/uc?id=1fgyvtdg6ShgXBvTZWQEBJGjKTjRATbSj
To: /content/ping_pong_dataset_40k.csv
100% 947k/947k [00:00<00:00, 42.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=11VhID0bod6neOfsUBkaYvkiEMVyr9Wjx
To: /content/ping_pong_dataset_bot.csv
100% 4.02M/4.02M [00:00<00:00, 48.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yZS6I7-j19HxVXuwD-2cOLv-SxDoGFsk
To: /content/ping_pong_dataset.csv
100% 563k/563k [00:00<00:00, 29.2MB/s]


In [ ]:
df = pd.read_csv('ping_pong_dataset_40k.csv')
df2 = pd.read_csv('ping_pong_dataset_bot.csv')
df3 = pd.read_csv('ping_pong_dataset.csv')
df = pd.concat([df,df2,df3])
df['action'] = (df.nothing)+(df.down*2)+(df.up*3)
df.action.value_counts()
# очистка данных
df = df[df.ball_x.between(np.percentile(df.ball_x, 1),
                          np.percentile(df.ball_x, 99))].copy()
df['diff_y'] = df.ball_y - df.paddle_y
# cond0 = df.ball_x>np.percentile(df.ball_x, 95)
# pdf = df[cond0].copy()
# diff_y_mean = pdf.diff_y.mean()
# diff_ystd = pdf.diff_y.std()
# cond1 = df.diff_y.between(-30, 30)
# mask = (cond0 & cond1) | (~cond0)
# df = df[mask].copy()
df.loc[(df.ball_x>350) & (np.abs(df.diff_y)<10), ['up','down','nothing','action']]=np.array([0,0,1,1])
df.loc[(df.ball_x>350) & (df.diff_y<-10), ['up','down','nothing','action']]=np.array([1,0,0,2])
df.loc[(df.ball_x>350) & (df.diff_y>10), ['up','down','nothing','action']]=np.array([0,1,0,3])

In [ ]:
# аугментация
df_inv = df.copy()
noise = np.random.normal(scale=5, size=df_inv.shape[0])
df_inv.ball_y = df_inv.ball_y.max() + df_inv.ball_y.min() - df_inv.ball_y + noise
df_inv.paddle_y = df_inv.paddle_y.max() + df_inv.paddle_y.min() - df_inv.paddle_y + noise
up = df_inv.down.values.copy()
down = df_inv.up.values.copy()
df_inv.up = up
df_inv.down = down
df_inv['action'] = (df_inv.nothing)+(df_inv.down*2)+(df_inv.up*3)
df_aug = pd.concat([df, df_inv])
# look-ahead
df_aug.up = df_aug.up.shift(-5)
df_aug.down = df_aug.down.shift(-5)
df_aug.nothing = df_aug.nothing.shift(-5)
df_aug = df_aug.dropna()

In [ ]:
# создание модели
inp = tf.keras.layers.Input((3,), name='input')
x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(inp)
x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
out = tf.keras.layers.Dense(3, activation='softmax', name='output', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
model = tf.keras.models.Model(inp, out)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
output (Dense)               (None, 3)                 195       
Total params: 17,667
Trainable params: 17,667
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# обучение
X = df_aug.loc[df_aug.ball_x>100, ['ball_x', 'ball_y', 'paddle_y']].values / 705 # нормализация
y = df_aug.loc[df_aug.ball_x>100, ['nothing', 'down', 'up']].values
tf.keras.backend.set_value(model.optimizer.lr, 2e-3)
model.fit(X,y,batch_size=128, epochs=10, verbose=1, class_weight={0:0.1, 1:1, 2:1})
tf.keras.backend.set_value(model.optimizer.lr, 2e-4) # понижаем шаг обучения
model.fit(X,y,batch_size=128, epochs=10, verbose=1, class_weight={0:0.1, 1:1, 2:1})

Epoch 1/10
539581/539581 [==============================] - 11s 21us/sample - loss: 0.2724 - acc: 0.7527
Epoch 2/10
539581/539581 [==============================] - 11s 20us/sample - loss: 0.2507 - acc: 0.7763
Epoch 3/10
539581/539581 [==============================] - 11s 20us/sample - loss: 0.2433 - acc: 0.7826
Epoch 4/10
539581/539581 [==============================] - 12s 23us/sample - loss: 0.2400 - acc: 0.7858
Epoch 5/10
539581/539581 [==============================] - 11s 20us/sample - loss: 0.2378 - acc: 0.7880
Epoch 6/10
539581/539581 [==============================] - 11s 20us/sample - loss: 0.2364 - acc: 0.7898
Epoch 7/10
539581/539581 [==============================] - 11s 21us/sample - loss: 0.2354 - acc: 0.7906
Epoch 8/10
539581/539581 [==============================] - 11s 20us/sample - loss: 0.2342 - acc: 0.7920
Epoch 9/10
539581/539581 [==============================] - 11s 20us/sample - loss: 0.2335 - acc: 0.7925
Epoch 10/10
539581/539581 [============================

In [ ]:
# проверка на оригинальных данных
confusion_matrix(df.loc[:, ['nothing', 'down', 'up']].values.argmax(-1),
                 model.predict(df[['ball_x', 'ball_y', 'paddle_y']].values / 705).argmax(axis=-1))

array([[98126,  7741, 27213],
       [  884, 86964, 18587],
       [ 1317,  1092, 49651]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df.loc[:, ['nothing', 'down', 'up']].values.argmax(-1), model.predict(df[['ball_x', 'ball_y', 'paddle_y']].values / 705).argmax(axis=-1)))

              precision    recall  f1-score   support

           0       0.98      0.74      0.84    133080
           1       0.91      0.82      0.86    106435
           2       0.52      0.95      0.67     52060

    accuracy                           0.81    291575
   macro avg       0.80      0.84      0.79    291575
weighted avg       0.87      0.81      0.82    291575



In [ ]:
# конвертация модели
onnx_model = onnxmltools.convert_keras(model, target_opset=9)

The ONNX operator number change on the optimization: 13 -> 9
The maximum opset needed by this model is only 7.


In [ ]:
# сохранение модели, тестовый запуск
with open('pingpong.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())
sess = onnxruntime.InferenceSession('pingpong.onnx')
print([(node.name, node.shape) for node in sess.get_inputs()])
sess.run(None, {'input': np.ones((1, 3), dtype=np.float32)})

[('input', ['N', 3])]


[array([[2.5585712e-05, 7.2734673e-03, 9.9270093e-01]], dtype=float32)]